In [17]:
import csv
import math
import numpy as np
from statsmodels.stats.stattools import durbin_watson
from scipy import stats
import statsmodels.formula.api as smf
import pandas as pd
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

In [3]:
datos=[]
with open('2022 MLB Player Stats - Batting.csv',newline='',encoding='utf-8') as archivo_csv:
    lector_csv=csv.DictReader(archivo_csv,delimiter=';')
    for fila in lector_csv:
        datos.append(fila)


#casteando la entrada
llaves=list(datos[0])
for columna in llaves:
    if not columna in ["Name", "Tm", "Lg"]:
        if not columna in ["BA", "OBP", "SLG", "OPS", "OPS+"]:
            for fila in datos:
                fila[columna]=int(fila[columna])
        else:
            for fila in datos:
                fila[columna]=float(fila[columna])

In [4]:
Over25_2D_NL=list(filter(lambda x: x['2B']>25 and x["Lg"]=="NL" , datos))

#El primer supuesto menciona que debe existir una relacion lineal entre las variables 
#dependientes y las independientes. Y solo hay dos variables que cumplen, por lo que al 
#aplicar el metodo foward no importa el orden en que tome la variable
#Las variables tomadas seran RBI y TB

Matriz=np.array(
    [
        [
            len(Over25_2D_NL),
            sum(map(lambda x:x['RBI'],Over25_2D_NL)),
            sum(map(lambda x:x['TB'],Over25_2D_NL))
        ],
        [
            sum(map(lambda x:x['RBI'],Over25_2D_NL)),
            sum(map(lambda x:x['RBI']*x['RBI'],Over25_2D_NL)),
            sum(map(lambda x:x['RBI']*x['TB'],Over25_2D_NL))
        ],
        [
            sum(map(lambda x:x['TB'],Over25_2D_NL)),
            sum(map(lambda x:x['TB']*x['RBI'],Over25_2D_NL)),
            sum(map(lambda x:x['TB']*x['TB'],Over25_2D_NL))
        ]
    ]
)

vector=np.array(
    [
        sum(map(lambda x:x['HR'],Over25_2D_NL)),
        sum(map(lambda x:x['RBI']*x['HR'],Over25_2D_NL)),
        sum(map(lambda x:x['TB']*x['HR'],Over25_2D_NL))
    ]   
)

c=np.linalg.solve(Matriz,vector)

In [5]:
print('comprobando los supuestos')

errores=list(map(lambda x:x['HR']-c[0]-c[1]*x['RBI']-c[2]*x['TB'],Over25_2D_NL))
NiveldeConfianza=0.05

#supuesto 1: Al escoger los campos que estaban en relacion lineal, tenemos el primer supuesto
print('Existe una relacion lineal entre las variables dependientes e independientes')

comprobando los supuestos
Existe una relacion lineal entre las variables dependientes e independientes


In [6]:

#supuesto de independencia de errores
dw=durbin_watson(errores)
print("El nivel de confianza es ",(1-NiveldeConfianza)*100,"%. Y el test de Durbin Watson da ",
    dw,". Por tanto los errores son ",("in" if dw>NiveldeConfianza else "")+'dependientes'
    )

El nivel de confianza es  95.0 %. Y el test de Durbin Watson da  2.2753518235513273 . Por tanto los errores son  independientes


In [7]:
#supuesto de suma y media de errores
if abs(sum(errores))<pow(10,-8):
    print('La suma de los errores es 0 y su media tambien es 0')
elif abs(sum(errores)/len(errores)<pow(10,-8)):
    print('La media de los errores es 0, pero su suma no')
else:
    print('Ni la media ni la suma de los errores es 0')


La suma de los errores es 0 y su media tambien es 0


In [12]:
#Supuesto de la normalidad
p1=0
n=len(Over25_2D_NL)
if n<50:
    stat1, p1 = stats.shapiro(errores)
    print("Como la muestra es menor que 50, se aplico el test de Shapiro-Wilk")
else:
    stat1, p1 =stats.kstest(errores)
    print("Como la muestra es mayor que 50, se aplico el test de Kolmologov Smirgol")
print("El resultado del Test de Shapiro-Wilk da:",p1,"por lo que los errores",
      "si" if p1>0.05 else "no", "estan normalmente distribuidos")

Como la muestra es menor que 50, se aplico el test de Shapiro-Wilk
El resultado del Test de Shapiro-Wilk da: 0.8901324272155762 por lo que los errores si estan normalmente distribuidos


In [22]:
dataframe = pd.DataFrame({'HR': list(map(lambda x:x['HR'],Over25_2D_NL)),
                          'RBI': list(map(lambda x:x['RBI']*c[1],Over25_2D_NL)),
                          'TB': list(map(lambda x:x['TB']*c[2],Over25_2D_NL)),
                          'C':list(map(lambda x:c[0],Over25_2D_NL))
                        })
fit = smf.ols('HR ~ RBI+TB+C', data=dataframe).fit()
print(fit.summary())
names = ['Lagrange multiplier statistic', 'p-value',
         'f-value', 'f p-value']
 
# Get the test result
test_result = sms.het_breuschpagan(fit.resid, fit.model.exog)
print("Se cumple el supuesto de Homocedasticidad")

lzip(names, test_result)

                            OLS Regression Results                            
Dep. Variable:                     HR   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     57.35
Date:                Wed, 21 Jun 2023   Prob (F-statistic):           4.61e-12
Time:                        01:50:34   Log-Likelihood:                -118.06
No. Observations:                  40   AIC:                             242.1
Df Residuals:                      37   BIC:                             247.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0706      0.021     -3.361      0.0

[('Lagrange multiplier statistic', 5.372675902469468),
 ('p-value', 0.14645547106649814),
 ('f-value', 2.87040673185525),
 ('f p-value', 0.06936500125821446)]